In [1]:
import pandas as pd
import xarray as xr
import numpy as np

In [2]:


# Load city data from CSV
cities_df = pd.read_csv('Data/cities_info.csv')

# Load the surface data file
fsurdat:str = "/home/jguo/projects/cesm/inputdata/lnd/clm2/surfdata_map/release-clm5.0.18/surfdata_0.9x1.25_hist_16pfts_Irrig_CMIP6_simyr2000_c190214.nc"
ds= xr.open_mfdataset(fsurdat)



In [3]:
ds

<xarray.Dataset>
Dimensions:                  (lsmlat: 192, lsmlon: 288, nlevsoi: 10,
                              natpft: 15, cft: 2, time: 12, lsmpft: 17,
                              numurbl: 3, numrad: 2, nlevurb: 5, nglcecp1: 11,
                              nglcec: 10)
Coordinates:
  * natpft                   (natpft) int32 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14
  * cft                      (cft) int32 15 16
  * time                     (time) int32 1 2 3 4 5 6 7 8 9 10 11 12
Dimensions without coordinates: lsmlat, lsmlon, nlevsoi, lsmpft, numurbl,
                                numrad, nlevurb, nglcecp1, nglcec
Data variables: (12/81)
    mxsoil_color             int32 ...
    SOIL_COLOR               (lsmlat, lsmlon) int32 dask.array<chunksize=(192, 288), meta=np.ndarray>
    PCT_SAND                 (nlevsoi, lsmlat, lsmlon) float64 dask.array<chunksize=(10, 192, 288), meta=np.ndarray>
    PCT_CLAY                 (nlevsoi, lsmlat, lsmlon) float64 dask.array<chunksize=(10, 192, 288), meta=np.ndarray>
    ORGANIC                  (nlevsoi, lsmlat, lsmlon) float64 dask.array<chunksize=(10, 192, 288), meta=np.ndarray>
    FMAX                     (lsmlat, lsmlon) float64 dask.array<chunksize=(192, 288), meta=np.ndarray>
    ...                       ...
    CONST_HARVEST_SH2        (lsmlat, lsmlon) float64 dask.array<chunksize=(192, 288), meta=np.ndarray>
    CONST_HARVEST_SH3        (lsmlat, lsmlon) float64 dask.array<chunksize=(192, 288), meta=np.ndarray>
    CONST_GRAZING            (lsmlat, lsmlon) float64 dask.array<chunksize=(192, 288), meta=np.ndarray>
    CONST_FERTNITRO_CFT      (cft, lsmlat, lsmlon) float64 dask.array<chunksize=(2, 192, 288), meta=np.ndarray>
    UNREPRESENTED_PFT_LULCC  (natpft, lsmlat, lsmlon) float64 dask.array<chunksize=(15, 192, 288), meta=np.ndarray>
    UNREPRESENTED_CFT_LULCC  (cft, lsmlat, lsmlon) float64 dask.array<chunksize=(2, 192, 288), meta=np.ndarray>
Attributes: (12/50)
    Conventions:                          NCAR-CSM
    History_Log:                          created on: 02-14-19 10:18:19
    Logname:                              erik
    Host:                                 cheyenne3
    Source:                               Community Land Model: CLM5
    Version:                              release-clm5.0.18/gpfs/fs1/scratch/...
    ...                                   ...
    map_vic_file:                         map_0.9x1.25_GRDC_to_0.9x1.25_nomas...
    Soil_texture_raw_data_file_name:      mksrf_soitex.10level.c010119.nc
    Soil_color_raw_data_file_name:        mksrf_soilcolor_CMIP6_simyr2005.c17...
    Fmax_raw_data_file_name:              mksrf_fmax_3x3min_USGS_c120911.nc
    Organic_matter_raw_data_file_name:    mksrf_organic_10level_5x5min_ISRIC-...
    Vegetation_type_raw_data_filename:    mksrf_landuse_histclm50_LUH2_2000.c...

In [6]:
print(ds['PCT_URBAN'])


<xarray.DataArray 'PCT_URBAN' (numurbl: 3, lsmlat: 192, lsmlon: 288)>
dask.array<open_dataset-PCT_URBAN, shape=(3, 192, 288), dtype=float64, chunksize=(3, 192, 288), chunktype=numpy.ndarray>
Dimensions without coordinates: numurbl, lsmlat, lsmlon
Attributes:
    long_name:  percent urban for each density type
    units:      unitless


In [7]:
# Access the 'PCT_URBAN' variable
pct_urban = ds['PCT_URBAN']

# Since pct_urban is a Dask array, compute the actual values
pct_urban = pct_urban.compute()

# Calculate the difference between the max and min values across the 'numurbl' dimension
diff = pct_urban.max(dim='numurbl') - pct_urban.min(dim='numurbl')

# Flatten the array of differences to make it easier to sort and find top values
flat_diff = diff.values.flatten()

# Get the indices of the top 10 differences
top_10_indices = np.argpartition(flat_diff, -10)[-10:]

# Convert these 1D indices back to 2D grid coordinates
top_10_coords = np.unravel_index(top_10_indices, diff.shape)

# Print out the top 10 differences and their corresponding grid coordinates
for i, index in enumerate(zip(*top_10_coords)):
    lat_index, lon_index = index
    print(f"Rank {i+1}: Grid Point ({lat_index}, {lon_index}) - Difference: {flat_diff[top_10_indices[i]]}")

Rank 1: Grid Point (140, 25) - Difference: 27.09479241727177
Rank 2: Grid Point (132, 28) - Difference: 43.81518761082444
Rank 3: Grid Point (107, 59) - Difference: 33.33333333333332
Rank 4: Grid Point (128, 25) - Difference: 31.461327851455174
Rank 5: Grid Point (135, 2) - Difference: 39.34353675450627
Rank 6: Grid Point (58, 243) - Difference: 29.00563643203609
Rank 7: Grid Point (104, 58) - Difference: 34.16811653321071
Rank 8: Grid Point (140, 218) - Difference: 27.468277706762738
Rank 9: Grid Point (59, 21) - Difference: 35.80042083526707
Rank 10: Grid Point (104, 63) - Difference: 41.666666666666664


In [8]:
# Access the 'PCT_URBAN' variable
pct_urban = ds['PCT_URBAN']

# Compute the actual values if it's a Dask array
pct_urban = pct_urban.compute()

# Calculate the difference between the max and min values across the 'numurbl' dimension
diff = pct_urban.max(dim='numurbl') - pct_urban.min(dim='numurbl')

# Flatten the array of differences to make it easier to sort and find top values
flat_diff = diff.values.flatten()

# Get the indices of the top 10 differences
top_10_indices = np.argpartition(flat_diff, -10)[-10:]

# Convert these 1D indices back to 2D grid coordinates
top_10_coords = np.unravel_index(top_10_indices, diff.shape)

# Print out the top 10 differences, their corresponding grid coordinates, and the values for all 3 slices
for i, index in enumerate(zip(*top_10_coords)):
    lat_index, lon_index = index
    # Access the PCT_URBAN values for all urban density types at the specific grid point
    urban_values = pct_urban.isel(lsmlat=lat_index, lsmlon=lon_index).values
    print(f"Rank {i+1}: Grid Point ({lat_index}, {lon_index}) - Difference: {flat_diff[top_10_indices[i]]}")
    for density_type, value in enumerate(urban_values):
        print(f"  Density Type {density_type}: {value}")


Rank 1: Grid Point (140, 25) - Difference: 27.09479241727177
  Density Type 0: 0.0
  Density Type 1: 8.26929592801137
  Density Type 2: 27.09479241727177
Rank 2: Grid Point (132, 28) - Difference: 43.81518761082444
  Density Type 0: 0.0
  Density Type 1: 4.830702498005143
  Density Type 2: 43.81518761082444
Rank 3: Grid Point (107, 59) - Difference: 33.33333333333332
  Density Type 0: 0.0
  Density Type 1: 0.0
  Density Type 2: 33.33333333333332
Rank 4: Grid Point (128, 25) - Difference: 31.461327851455174
  Density Type 0: 0.0
  Density Type 1: 2.75669975618301
  Density Type 2: 31.461327851455174
Rank 5: Grid Point (135, 2) - Difference: 39.34353675450627
  Density Type 0: 0.0
  Density Type 1: 16.26808267472303
  Density Type 2: 39.34353675450627
Rank 6: Grid Point (58, 243) - Difference: 29.00563643203609
  Density Type 0: 0.0
  Density Type 1: 29.00563643203609
  Density Type 2: 11.573350781474788
Rank 7: Grid Point (104, 58) - Difference: 34.16811653321071
  Density Type 0: 0.0
 

#  Access the required variables from the netCDF file

In [ ]:
longitude = ds['LONGXY'].values
latitude = ds['LATIXY'].values
pct_urban = ds['PCT_URBAN'].sum(dim='numurbl')  # Assuming PCT_URBAN needs to be summed across urban types

In [ ]:


# Prepare longitude and latitude grids for broadcasting
lon_grid, lat_grid = np.meshgrid(longitude[0, :], latitude[:, 0])  # Adjust if longitude and latitude are 1D

In [ ]:

# Reshape city coordinates for broadcasting
city_lons = cities_df['Lon'].values[:, np.newaxis, np.newaxis]
city_lats = cities_df['Lat'].values[:, np.newaxis, np.newaxis]

In [ ]:

# Compute absolute differences using broadcasting
lon_diff = np.abs(lon_grid - city_lons)
lat_diff = np.abs(lat_grid - city_lats)

In [ ]:

# Find the indices of the grid cell closest to each city
lon_idx = np.argmin(lon_diff, axis=2)
lat_idx = np.argmin(lat_diff, axis=1)

In [ ]:

# Initialize list to store matched data
matched_data = []

# Iterate over the cities using the index
for city_index in range(len(cities_df)):
    lon_i = lon_idx[city_index]  # Longitude index for the city
    lat_i = lat_idx[city_index]  # Latitude index for the city

    # Extract the urban value and ensure it's loaded as a NumPy array
    urban_value = pct_urban.isel(lsmlat=lat_i, lsmlon=lon_i).values

    # If you need a scalar value and you're sure it's a single value, you can use .item()
    if urban_value.size == 1:
        urban_value = urban_value.item()

    # Append matched data
    matched_data.append({
        'LONGXY': longitude[0, lon_i],
        'LATIXY': latitude[lat_i, 0],
        'PCT_URBAN': urban_value,
        'Name': cities_df.iloc[city_index]['Name'],
        'Lat': cities_df.iloc[city_index]['Lat'],
        'Lon': cities_df.iloc[city_index]['Lon'],
        'lsmlon': lon_i,
        'lsmlat': lat_i
    })


# Convert matched data to DataFrame
matched_df = pd.DataFrame(matched_data)

# Display the DataFrame
matched_df